In [0]:
import requests
import csv
import numpy as np
import pandas as pd
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load Data

In [0]:
COLS = ['cuisine', 'title', 'ingredients', 'process', 'instructions', 'keto', 'healthScore', 'diets', 'nutritions']
data = pd.read_csv('/content/drive/My Drive/Study @ Fordham/NLP Project/nlp_finalproj_data_temp.csv')


In [4]:
data['cuisine'].value_counts()

Middle Eastern    200
French            200
Mediterranean     200
Vietnamese        200
Chinese           200
Italian           200
African           123
Indian             99
Name: cuisine, dtype: int64

# Encode and seperate features

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re

from sklearn import preprocessing


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
def deduplicate(ingredient):
    # try to uniform the duplicated or similar ingredients
    ingredient = re.sub('canned |fresh |cooked |dried |dry |ground |refrigerated |light |ready-to-use ', '', ingredient)
    ingredient = re.sub('lettuc[^0-9]+', 'lettuc', ingredient)
    ingredient = re.sub('[^0-9]+ salt', 'salt', ingredient)
    ingredient = re.sub('[^0-9]+ basil|basil [^0-9]+', 'basil', ingredient)
    ingredient = re.sub('[^0-9]+ bacon|bacon [^0-9]+', 'bacon', ingredient)
    ingredient = re.sub('[^0-9]+ thyme|thyme [^0-9]+', 'thyme', ingredient)
    ingredient = re.sub('[^0-9]+ cilantro|cilantro [^0-9]+', 'cilantro', ingredient)
    ingredient = re.sub('[^0-9]+ parsley|parsley [^0-9]+', 'parsley', ingredient)
    ingredient = re.sub('[^0-9]+ cinnamon|cinnamon [^0-9]+', 'cinnamon', ingredient)
    ingredient = re.sub('cream[^0-9]+chees|cream chees[^0-9]+', 'cream chees', ingredient)
    ingredient = re.sub('[^0-9]+parmesan[^0-9]+', 'parmesan chees', ingredient)
    ingredient = re.sub('[^0-9]+ peanut butt', 'peanut butt', ingredient)
    ingredient = re.sub('[^0-9]+ chili pepper', 'chili pepp', ingredient)
    ingredient = re.sub('[^0-9]+ nectar', 'nectar', ingredient)
    ingredient = re.sub('[^0-9]+ tofu', 'tofu', ingredient)
    ingredient = re.sub('[^0-9]+ tomato', 'tomato', ingredient)
    ingredient = re.sub('[^0-9]+ bell pepp', 'bell pepp', ingredient)
    ingredient = re.sub('egg roll wra[^0-9]+', 'egg roll wrap', ingredient)
    return ingredient


def clean_data(igd_data):
    # igd_data: a array-like object which contains all ingredients list of training dataset
    # stem: boolean, default to be True
    # *output: a dict
    igds = list()
    raw_igds = igd_data.split(sep=',')
    for i in raw_igds:
        igds.append(re.sub(' *\'|\[|\]|\ *"', '', i)) # delete unneeded punctuations
    return igds

def stem_and_deduplicate(igd_data):
    ps = PorterStemmer()
    stem_igds = [ps.stem(x) for x in igd_data]
    return [deduplicate(y) for y in stem_igds]

In [0]:
# Split and encode ingredients

all_ingredients = [stem_and_deduplicate(clean_data(x)) for x in data['ingredients']]

le_igd = preprocessing.LabelEncoder()
le_igd.fit(np.hstack(all_ingredients))

data['encoded_ingredients'] = [le_igd.transform(x) for x in all_ingredients]



In [0]:
# Concatenate the ingredients and process

# Split the process
ps = PorterStemmer()
process = list()
for i in range(len(data)):
    tem = data['process'][i].split(sep=', [')
    tokens = [re.sub(' *\'|\[|\]|\ *"', '', i).replace(',', ' ') for i in tem]
    process.append([ps.stem(x) for x in tokens])

# Link them
process_igds = []
for (n, left) in enumerate(process):
    right = all_ingredients[n]
    piece = []
    for (i, p) in enumerate(left):
        piece.append(p+' '+right[i])
    process_igds.append(piece)

data['process ingredients'] = process_igds

In [0]:
def extract_methods(methods_data, stopwords):
    tokens = word_tokenize(str(methods_data).lower())
    tagged_ins = nltk.pos_tag(tokens)
    methods = set()
    ps = PorterStemmer()
    for (m, tag) in tagged_ins:
        if tag in ['VB']: # NOT decided if to take NN in
            methods.add(ps.stem(m))
    effective_methods = [w for w in methods if not w in stopwords]
    return effective_methods

In [0]:
# Split and encode methods (from instructions)

freq_vb = ['put', 'add', 'use', 'keep', 'prepar', 'start']

data['instructions'] = [extract_methods(x, freq_vb) for x in data['instructions']]

# Encode the instructions words
le_methods = preprocessing.LabelEncoder()
le_methods.fit(np.hstack(data['instructions']))

data['encoded_instructions'] = [le_methods.transform(x) for x in data['instructions']]


In [0]:
# Split and encode diets

all_diets = [stem_and_deduplicate(clean_data(x)) for x in data['diets']]

le_diets = preprocessing.LabelEncoder()
le_diets.fit(np.hstack(all_diets))
le_diets.classes_

data['encoded_diets'] = [le_diets.transform(x) for x in all_diets]

# Dict for search
code2diets = {x:n for n,x in zip(range(10),le_diets.classes_.tolist())}

diet_info = np.zeros((len(data), 11))
for i in le_diets.classes_.tolist():
    for (num, recipe_info) in enumerate(data['encoded_diets']):
        for diet in recipe_info:
            diet_info[int(num), int(diet)] = 1

diet_names = le_diets.classes_.tolist()
diet_names[0] = 'no diet'

diet_info = pd.DataFrame(diet_info, columns=diet_names)
data = data.join(diet_info)


In [0]:
# Seperate nutrition feature into three columns

tem = []
for n in data['nutritions']:
    numberonly = re.sub(' *\'|\{|\}|\ *"|[a-z|A-Z]|:', '', n)
    ns = re.split(r',', numberonly)
    tem.append(ns)

data['protein'], data['fat'], data['carbs'] = [float(x[0]) for x in tem], [float(x[1]) for x in tem], [float(x[2]) for x in tem]


In [0]:
# Save the preprocessed data

data.to_csv('nlp_finalproj_data_preprocessed.csv')